---
  title: "RestFull 04: Passage de paramètres"
  description: "parameters with Jakarta RESTful Web Services."
  categories: 
    - Java
    - Lecture
    - RestFull
  provide_notes: true
  provide_slides: false
  jupyter: java-lts
  echo: true
  output: true  
---

{{< embed ./quarto-utils/_version.qmd >}}

## Les paramètres complexes dans les corps de requêtes 

Les conversions sont aussi automatiques dans l'autre sens quand des données (XML, JSON, ...) sont envoyées dans le corps d'une requête HTTP (par exemple PUT ou POT) pour être utilisée simplement comme un parametre Java de la méthode. Les méthodes qui nécessitent des paramètres sont annotées avec `@Consumes(type[, type, …])` pour indiquer les types mime supportés. 

L'annotation `@Consummes` de la méthode `addAuthor()` ci-dessous indique ce qui est possible et l'entete http `Content-Type` de la requête ce qui est envoyé. Noter que l'Id n'est pas indiqué dans la requête mais que l'entité complète est retournée. 

In [1]:
//| output: false
//| echo: false

%printWithName false
    
// UPDATE SAMPLE SOURCE CODE

String script="""
GITHUB_REPO=ebpro/sample-jaxrs
GITHUB_URL=https://github.com/${GITHUB_REPO}
BRANCH=develop
SRC_DIR=/home/jovyan/work/src/github/${GITHUB_REPO}
gitpuller ${GITHUB_URL} ${BRANCH} ${SRC_DIR}
cd ${SRC_DIR}
mvn --quiet clean package
""";
IJava.getKernelInstance().getMagics().applyCellMagic("shell",List.of(""),script);  

$ git fetch


$ git reset --mixed


$ git -c user.email=nbgitpuller@nbgitpuller.link -c user.name=nbgitpuller merge -Xours origin/develop


Already up to date.


0

In [2]:
//| output: false
//| echo: false
%jars "/home/jovyan/work/src/github/ebpro/sample-jaxrs/target/sample-jaxrs-*-withdependencies.jar"; 
import org.glassfish.grizzly.http.server.HttpServer;
import fr.univtln.bruno.samples.jaxrs.server.BiblioServer;
HttpServer httpServer = BiblioServer.startServer();
httpServer.toString();

org.glassfish.grizzly.http.server.HttpServer@3751eff4

In [3]:
//| output: true
//| echo: false
// PRINT CLASS
String script="/home/jovyan/work/src/github/ebpro/sample-jaxrs/src/main/java/fr/univtln/bruno/samples/jaxrs/resources/AuthorResource.java";
IJava.getKernelInstance().getMagics().applyCellMagic("javasrcMethodByName",List.of("AuthorResource","addAuthor"),script);
return null;

```Java
/**
 * Adds an new author to the data.
 * Status annotation is a trick to fine tune 2XX status codes (see the status package).
 *
 * @param author The author to be added without its id.
 * @return The added author with its id.
 * @throws IllegalArgumentException if the author has an explicit id (id!=0).
 */
@POST
@Status(Status.CREATED)
@Consumes(MediaType.APPLICATION_JSON)
public Library.Author addAuthor(Library.Author author) throws BusinessException {
    return Library.demoLibrary.addAuthor(author);
}
```

Adds an author :

In [4]:
%%shell
curl -s -i -H "Accept: application/json"  \
  -H "Content-type: application/json"  \
  -X POST \
  -d '{"name":"John","firstname":"Smith","biography":"My life"}' \
   http://localhost:9998/mylibrary/authors

HTTP/1.1 201 Created


Content-Type: application/json


Content-Length: 64


{"id":1,"name":"John","firstname":"Smith","biography":"My life"}


La méthode `updateAuteur` est appelée par un PUT mais avec une resource précise (indiquée dans l'URL) à mettre à jour.

In [5]:
//| output: true
//| echo: false
// PRINT CLASS
String script="/home/jovyan/work/src/github/ebpro/sample-jaxrs/src/main/java/fr/univtln/bruno/samples/jaxrs/resources/AuthorResource.java";
IJava.getKernelInstance().getMagics().applyCellMagic("javasrcMethodByName",List.of("AuthorResource","updateAuthor"),script);
return null;

```Java
/**
 * Update an author with an given id.
 *
 * @param id     the id injected from the path param "id"
 * @param author a injected author made from the JSON data (@Consumes) from body of the request. This author is forbidden to havce an Id.
 * @return The resulting author with its id.
 * @throws NotFoundException        is returned if no author has the "id".
 * @throws IllegalArgumentException is returned if an "id" is also given in the request body.
 */
@PUT
@Path("{id}")
@Consumes(MediaType.APPLICATION_JSON)
public Library.Author updateAuthor(@PathParam("id") long id, Library.Author author) throws BusinessException {
    return Library.demoLibrary.updateAuteur(id, author);
}
```

Fully update an author

In [6]:
%%shell
curl -s -i -H "Accept: application/json"  \
  -H "Content-type: application/json"  \
  -X PUT \
  -d '{"name":"Martin","firstname":"Jean","biography":"ma vie"}' \
  http://localhost:9998/mylibrary/authors/1

HTTP/1.1 200 OK


Content-Type: application/json


Content-Length: 64


{"id":1,"name":"Martin","firstname":"Jean","biography":"ma vie"}


### La suppression
La suppression des ressources se fait avec les approches précédentes.

In [7]:
//| output: true
//| echo: false
// PRINT CLASS
String script="/home/jovyan/work/src/github/ebpro/sample-jaxrs/src/main/java/fr/univtln/bruno/samples/jaxrs/resources/AuthorResource.java";
IJava.getKernelInstance().getMagics().applyCellMagic("javasrcMethodByName",List.of("AuthorResource","removeAuthor"),script);
return null;


```Java
/**
 * Removes an author by id from the data.
 *
 * @param id the id of the author to remove
 * @throws NotFoundException is returned if no author has the "id".
 */
@DELETE
@Path("{id}")
public void removeAuthor(@PathParam("id") final long id) throws BusinessException {
    Library.demoLibrary.removeAuthor(id);
}
```

Removes one author :

In [8]:
%%shell
curl -s -i -X DELETE \
    http://localhost:9998/mylibrary/authors/1

HTTP/1.1 204 No Content


In [9]:
//| output: true
//| echo: false
// PRINT CLASS
String script="/home/jovyan/work/src/github/ebpro/sample-jaxrs/src/main/java/fr/univtln/bruno/samples/jaxrs/resources/AuthorResource.java";
IJava.getKernelInstance().getMagics().applyCellMagic("javasrcMethodByName",List.of("AuthorResource","removeAuthors"),script);
return null;


```Java
/**
 * Removes every authors
 */
@DELETE
public void removeAuthors() {
    Library.demoLibrary.removesAuthors();
}
```

Removes all authors

In [10]:
%%shell
curl -s -i -X DELETE \
    http://localhost:9998/mylibrary/authors/

HTTP/1.1 204 No Content


In [11]:
//| output: false
//| echo: false
httpServer.stop();